Realizar conexion a la API

In [1]:
import requests
import pandas as pd
import time

url = 'https://www.buda.com/api/v2/markets'
response = requests.get(url)

df_mercado = pd.DataFrame(response.json()['markets'])
df_mercado['name'].unique()

array(['btc-clp', 'btc-cop', 'eth-clp', 'eth-btc', 'btc-pen', 'eth-pen',
       'eth-cop', 'bch-btc', 'bch-clp', 'bch-cop', 'bch-pen', 'btc-ars',
       'eth-ars', 'bch-ars', 'ltc-btc', 'ltc-clp', 'ltc-cop', 'ltc-pen',
       'ltc-ars', 'usdc-clp', 'usdc-cop', 'usdc-pen', 'usdc-ars',
       'btc-usdc', 'usdt-usdc'], dtype=object)

1. Calcular spread de cualquiera de los mercados de Buda.com.

In [2]:
market_id = 'btc-clp'
url_market = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
def spread_market(market_id):
    response = requests.get(url_market)
    if response.status_code == 200:
        data = response.json()
        spread = float(data['ticker']['min_ask'][0]) - float(data['ticker']['max_bid'][0])
        return spread
    else:
        print(f"ERROR: No se puede obtener datos.")
        return 0

df_spread = spread_market(market_id)
print(f"Spread para {market_id}: {df_spread}")


Spread para btc-clp: 227544.0


2. Obtener el spread de todos los mercados en una sola llamada.

In [3]:
def spread_mercados():

    markets_url = 'https://www.buda.com/api/v2/markets'
    markets_response = requests.get(markets_url)
    
    if markets_response.status_code == 200:
        markets_data = markets_response.json()
        spreads = {}

        for mercado in markets_data.get('markets', []):
            market_id = mercado.get('id')
            if market_id:

                ticker_url = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
                ticker_response = requests.get(ticker_url)
                
                if ticker_response.status_code == 200:
                    ticker_data = ticker_response.json().get('ticker', {})
                    
                    if 'min_ask' in ticker_data and 'max_bid' in ticker_data:

                        min_ask = float(ticker_data['min_ask'][0])
                        max_bid = float(ticker_data['max_bid'][0])
                        spread = min_ask - max_bid
                        spreads[market_id] = spread
                else:
                    print("ERROR: No se puede obtener datos del ticker.")
                    
        return spreads
    else:
        print("ERROR: No se puede obtener datos de los mercados.")
        return None


spreads_mercados = spread_mercados()

if spreads_mercados:
    print("Lista de spreads:")
    for market_id, spread in spreads_mercados.items():
        print(f"{market_id}: {spread}")
else:
    print("ERROR: No se puede obtener la lista.")



Lista de spreads:
BTC-CLP: 227544.0
BTC-COP: 1827727.0
ETH-CLP: 24217.01000000001
ETH-BTC: 0.0007917399999999991
BTC-PEN: 3535.899999999994
ETH-PEN: 279.35000000000036
ETH-COP: 270104.0
BCH-BTC: 0.0005899
BCH-CLP: 2346.890000000014
BCH-COP: 35170.0
BCH-PEN: 11.129999999999995
BTC-ARS: 0.0
ETH-ARS: 0.0
BCH-ARS: 0.0
LTC-BTC: 3.366000000000003e-05
LTC-CLP: 343.0
LTC-COP: 8406.090000000026
LTC-PEN: 2.3100000000000023
LTC-ARS: 0.0
USDC-CLP: 4.610000000000014
USDC-COP: 39.0
USDC-PEN: 0.020000000000000018
USDC-ARS: 0.0
BTC-USDC: 587.2200000000012
USDT-USDC: 0.0005999999999999339


3. Spread alerta

In [7]:
def obtener_alerta():
    try:
        with open('spread_alerta.txt', 'r') as file:
            spread_alerta = float(file.read())
        return spread_alerta
    except FileNotFoundError:
        return None
    except Exception as e:
        print(f"Error al leer: {e}")
        return None

def guardar_alerta(ejemplo_market,limite):
    try:
        with open('spread_alerta.txt', 'w') as file:
            file.write(str(limite))
        print(f"Spread de alerta guardado para {ejemplo_market}: alerta de {limite}.")
    except Exception as e:
        print(f"Error al guardar: {e}")

def realizar_polling(market_id,tiempo):
    while True:
        spread_actual = spread_market(market_id)
        spread_alerta = obtener_alerta()

        if spread_actual is not None and spread_alerta is not None:
            if spread_actual > spread_alerta:
                print(f"¡Alerta! El spread actual ({spread_actual}) es mayor que el spread de alerta ({spread_alerta}).")
            elif spread_actual < spread_alerta:
                print(f"¡Alerta! El spread actual ({spread_actual}) es menor que el spread de alerta ({spread_alerta}).")

        time.sleep(tiempo)

# Ejemplo de uso
ejemplo_market = 'btc-clp'
limite = 200000
tiempo=60

guardar_alerta(ejemplo_market,limite)
realizar_polling(ejemplo_market,tiempo)


Spread de alerta guardado para btc-clp: alerta de 200000.
¡Alerta! El spread actual (226942.0) es mayor que el spread de alerta (200000.0).
